# sayitinsaami.yle.fi scraper

> "For a DT2112 project"

- toc: false
- branch: master
- hidden: true
- categories: [sami, scraper, dt2112]

In [1]:
JSON = "http://sayitinsaami.yle.fi/js/fraasit.js?2019-02-15"

In [2]:
import requests

In [3]:
rawdata = requests.get(JSON)

In [4]:
to_clean = rawdata.text
to_clean = to_clean.replace(" f:", ' "f":').replace(" fi:", ' "fi":').replace(" sv:", ' "sv":').replace(" en:", ' "en":').replace(" sme:", ' "sme":').replace(" smn:", ' "smn":').replace(" sms:", ' "sms":').replace(" cats:", ' "cats":')

In [5]:
if to_clean.startswith("FRAASIT = "):
    to_clean = to_clean[10:]
to_clean = to_clean.strip()
if to_clean.endswith("},\n]"):
    to_clean = to_clean.replace("},\n]", "}]")


In [6]:
import json
data = json.loads(to_clean)

In [8]:
with open("output.tsv", "w") as outf:
    for item in data:
        outf.write(f'{item["sme"]}\thttp://sanosesaameksi.katrikoivula.com/audio/sme/{item["f"]}.m4a\n')

In [13]:
!cat output.tsv|awk -F'\t' '{print $2}'|while read i;do f=$(echo $i|awk -F/ '{print $NF}'); if [ ! -e $f ];then wget -c $i;fi;done

In [14]:
with open("sayitinsaami.json", "w") as cleaned:
    json.dump(data, cleaned)

In [ ]:
!ls *.m4a|zip sayitinsaami_audio.zip -@

In [ ]:
!for f in *.m4a;do ffmpeg -i $f  -acodec pcm_s16le -ac 1 -ar 16000 $(basename $f .m4a).wav;done

In [ ]:
!ls -al *.wav

In [32]:
text = []
audio = []
for item in data:
    text.append(item["sme"])
    audio.append(f'{item["f"]}.wav')

In [20]:
%%capture
!pip install datasets

In [28]:
%%capture
!pip install torchaudio librosa

In [23]:
from datasets import Dataset, Audio

In [33]:
audio_ds = Dataset.from_dict({"text": text, "audio": audio}).cast_column("audio", Audio())

In [ ]:
audio_ds[1]